# Install Requirements and Import Libraries

In [ ]:
!pip install evaluate seqeval -qqq # Requirements for the kaggle

In [ ]:
from huggingface_hub import login
import wandb
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import evaluate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np

# Authenticate with Wandb Log and Huggingface

In [ ]:
wandb.login(key="")
login(token="")

# Load the DB and Models

In [ ]:
ds = load_dataset("chuuhtetnaing/myanmar-text-segmentation-dataset")
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

seqeval = evaluate.load("seqeval")

In [ ]:
id2label = {i: l for i, l in enumerate(ds['train'].features['segment_tags'].feature.names)}
label2id = {l: i for i, l in enumerate(ds['train'].features['segment_tags'].feature.names)}

In [ ]:
id2label

In [ ]:
label2id

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

# Evaluaiton Function and Tokenization Function

In [ ]:
label_list = ds["train"].features[f"segment_tags"].feature.names

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"segment_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)

In [ ]:
tokenized_ds

# Train/Fine-Tune the Model

In [ ]:
training_args = TrainingArguments(
    output_dir="myanmar_text_segmentation_model",
    learning_rate=2e-5,
    per_device_train_batch_size=25,
    per_device_eval_batch_size=25,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=True,
    report_to="wandb",
    hub_private_repo=True,
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    hub_strategy="all_checkpoints",
    save_safetensors=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()